In [1]:
import re,requests
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from os import listdir
from os.path import isfile, join

In [2]:
words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х', 'a', 'd', 'b'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)
stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
        'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже', 'forbes.kz',
       'I','II','III','т']


In [3]:
s = requests.Session()
bolezni = pd.read_csv('bolezni.csv')
# bolezni['name']

In [4]:
url = 'http://www.eurolab.ua/diseases'
urls = {}
page = s.get(url)
soup = BeautifulSoup(page.content, 'lxml')
mu = soup.find('div', {'align': 'center'})
mu = mu.find_all('a')
for i in mu:
    page = s.get('http://www.eurolab.ua'+ i['href'])
    soup = BeautifulSoup(page.content, 'lxml')
    su = soup.find('div', {'class': 'l-main'})
    while su.find('div'):
        su.find('div').decompose()
    su = su.find_all('a')
    for j in su:
        urls[j.text]='http://www.eurolab.ua'+j['href']
    soup = BeautifulSoup(page.content, 'lxml')
    while soup.find('div',{'class':'paginationControl'}):
        nxt = soup.find('div',{'class':'paginationControl'})
        aas = nxt.find_all('a')
        if aas[len(aas)-1]['title']=='Назад':
            url = 'http://www.eurolab.ua'+ aas[-1]['href']
            page = s.get(url)
            soup = BeautifulSoup(page.content, 'lxml')
            su = soup.find('div', {'class': 'l-main'})
            while su.find('div'):
                su.find('div').decompose()
            su = su.find_all('a')   
            for j in su:
                urls[j.text]='http://www.eurolab.ua'+j['href']
        else:
            break
        soup = BeautifulSoup(page.content, 'lxml')
            
    
len(urls)

2607

In [5]:
texts = []
for i in urls:
    page = s.get(urls[i])
    soup = BeautifulSoup(page.content, 'lxml')
    mu = soup.find('div', {'class': 'l-main'})
#     try:
#         if mu.find('div',{'class':'like_p with-sign'}):
#             mu.find('div',{'class':'like_p with-sign'}).decompose()
#         su = mu.find_all('div',{'class':'like_p'})
# #     except:
# #         texts.append('')
# #         continue
    su = mu.find_all('div',{'rounded'})
    
#     [j.find('div',{'class':'like_p with-sign'}).decompose() for j in su if j.find('div',{'class':'like_p with-sign'})]
    text =''
    for j in su[:len(su)-2]:
        text += j.text
        
    for j in stop:
        text = text.replace(j, '')
        text = text.replace('?', '.')
        text = re.sub('\d+', ' ', text)
        text = re.sub('[^\w.]', ' ', text)

        final_s = ''
        sentences = text.split('.')
        for sen in sentences:
            sen = sen.strip()
            for tok in sen.split(' '):
                tok = tok.strip()
                if(tok.lower() not in set_stopWords):
                    final_s += (tok + ' ')
            final_s += '.'

    texts.append(final_s)


In [6]:
uri = []
name =[]
for i in urls:
    uri.append(urls[i])
    name.append(i)

In [7]:
res = pd.DataFrame()
res['name'] = name
res['url'] = uri
res['descr'] = texts
res.head()
res.to_csv('bolezni_eurolab.csv', index=False)

In [8]:
a = set()
num = 0
for i in bolezni['name']:
    if i in urls:
          num += 1